### Get link

Doc: https://developers.google.com/youtube/v3/docs/search/list?hl=vi

In [34]:
strs = ['AIza','SyCXm','llW1PCC','yoOq','Ls5j2Db','ZBltp9','EFqWzI']
api_key = ''.join(strs)

In [35]:
# limit time query
from datetime import datetime
from dateutil.relativedelta import relativedelta

def get_previous_date(month, day):
    today = datetime.now()

    # Lấy ngày của tháng trước
    previous_month_date = today - relativedelta(months=month, days=day)

    # Chuyển đổi sang định dạng yêu cầu "YYYY-MM-DDTHH:MM:SSZ"
    formatted_date = previous_month_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    return formatted_date


In [36]:
from googleapiclient.discovery import build

# Init YouTube API client
youtube = build('youtube', 'v3', developerKey=api_key)

def search_youtube(query, date, max_results=150):
    request = youtube.search().list(
        q=query,
        part="snippet",
        type="video",
        order="date", # sort: date, rating, relevance, viewCount
        publishedAfter=date, #date
        maxResults=max_results
    )
    response = request.execute()
    return response['items']
        

In [37]:
query = "samsung+kg+mdm+unlock"
previous_date = get_previous_date(month=0, day=30)
print(previous_date)

search_result = search_youtube(query= query, date=previous_date)

2024-08-13T13:20:09Z


In [38]:
#ignore title
ignore_word = ["T Mobile", "US Cellular", "Sprint USA", "Unlock Service", "Xfinity USA", "Cricket USA", "FRP", "Boost USA", "Verizon USA", "Spectrum", "Lost mode", "Huawei",
                   "Xiaomi", "screen lock", "TFN", "iphone", "icloud"]

In [39]:
video_ids = [item['id']['videoId'] for item in search_result]

### Get data API

In [40]:
# get video id
from urllib.parse import urlparse, parse_qs

def get_video_id(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    return query_params.get('v', [None])[0]

# Ví dụ
url = 'https://www.youtube.com/watch?v=dQw4w9WgXcQ'
video_id = get_video_id(url)
print(f'Video ID: {video_id}')

Video ID: dQw4w9WgXcQ


In [41]:
#print json
import json

def prinJson(data):
    formatted_json = json.dumps(data, indent=4, ensure_ascii=False)
    print(formatted_json)

In [42]:
import requests

def getVideoInfo(video_id):
    # URL để gọi YouTube Data API
    url = f'https://www.googleapis.com/youtube/v3/videos?id={video_id}&key={api_key}&part=snippet,contentDetails,statistics'

    # GET
    response = requests.get(url)
    # check success
    if response.status_code == 200:
        data = response.json()
        video_info = data['items'][0]
        return video_info
    else:
        print(f'Error: {response.status_code}')

def getData(link, video_info):
    title = video_info['snippet']['title']
    description = video_info['snippet']['description']
    published_at = video_info['snippet']['publishedAt']
    return ['youtube', link, published_at, title,  description]

In [46]:
columns = ['Type', 'Link',  'Published', 'Title', 'Content']
data = []
for i, video_id in enumerate(video_ids):
    video_info = getVideoInfo(video_id)
    link = f"https://www.youtube.com/watch?v={video_id}"
    data_row = getData(link, video_info)
    print(data_row[3])
    data.append(data_row)

Samsung Frp Bypass And KG Unlock / MDM Remove FREE Tool | Support Android Version 10/11/12/13/14
Samsung mdm remove tool | Samsung KG Unlock And MDM Lock/Knox Remove Tool
Samsung Z Fold 6 KG Lock Remove Parmanent ||  all software online cont watsap_  9024280280
all Samsung kg lock bypass permanent new security 2024 / all Samsung finance lock bypass new securit
MdmFixTool Samsung KG MDM LOCK ALL Android 14 devices ADB Mode support for MTK, Exynos, Qualcomm SPD
Samsung a53 kg lock permanently unlock MDM can't download mode pit error fix
Samsung MDM Knox Remove 2024 tool
SAMSUNG KG LOCK REMOVE NEW SECURITY ANDROID 14 2014 || Samsung A14 U6 KG Lock Remove EASY JTAG
Android 14 KGG QC Kg downgrade Android 13 🔓 Samsung Android 14 KGG Remove done ✅ permanent 2024 mdm🔥
Samsung Android 14 Fail to load KG please download official Samsung Android 14 kg lost mode fix 2024
Samsung A14 kg unlock  sm-a146b  Android 14 Latest Security Octoplus Samsung Tool
Samsung A54 kg mdm lock unlock parmannent by c

In [44]:
import pandas as pd
import os
from datetime import datetime
today = datetime.today().date()

df = pd.DataFrame(data, columns=columns)

file_path = f'../output/output_{today}.xlsx'
if os.path.exists(file_path):
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
        df.to_excel(writer, sheet_name=f'youtube_{today}', index=False)
else:
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
        df.to_excel(writer, sheet_name=f'youtube_{today}', index=False)

In [45]:
print('Success')

Success


query"
("Samsung Knox Guard" OR "Samsung MDM" OR "Samsung KG") AND ("unlock" OR "unfasten" OR "unbolt" OR "open" OR "release" OR "unlatch" OR "disengage" OR "free" OR "unseal" OR "uncover" OR "access") AND ("bypass" OR "circumvent" OR "avoid" OR "sidestep" OR "evade" OR "skip" OR "dodge" OR "work around" OR "ignore" OR "overcome" OR "elude") AND ("removal" OR "elimination" OR "deletion" OR "eradication" OR "extraction" OR "withdrawal" OR "dismissal" OR "expulsion" OR "displacement" OR "ouster" OR "exclusion") AND ("tool" OR "software" OR "method" OR "technique" OR "unlocker" OR "key generator" OR "exploit" OR "vulnerability" OR "APK") AND ("ADB" OR "flash firmware") AND ("guide" OR "tutorial" OR "step-by-step" OR "how-to") AND ("legal" OR "issues" OR "compatibility" OR "support") AND ("community forums" OR "troubleshooting") AND ("2024" OR "updated methods") AND ("Galaxy S-series" OR "Note-series" OR "latest security patch") AND (date:2024-08)

32 words only for google search:
("Samsung Knox Guard" OR "Samsung MDM" OR "Samsung KG") AND ("unlock" OR "bypass" OR "removal") AND ("tool" OR "method" OR "software" OR "guide") AND ("August 2024" OR "latest update")
